In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import ast
%matplotlib inline

In [158]:
import pickle

In [125]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [126]:
movies = movies.merge(credits,on='title')

In [127]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [128]:
def get_genre_keywords(text):
    lis = ""
    for i in ast.literal_eval(text):
        lis = lis + " "+ (i['name'].lower()) 
    return lis

In [129]:
def get_top_cast(text):
    lis = []
    for i in ast.literal_eval(text):
        lis.append(i['name'])
    lis = [string.lower().replace(" ","") for string in lis]
    return " ".join(lis[:3])

In [130]:
def get_director(text):
    lis = []
    for i in ast.literal_eval(text):
        if i["job"] == "Director":
            return i['name'].lower().replace(" ","")   

In [131]:
movies['cast'] = movies['cast'].apply(get_top_cast)

In [132]:
movies['crew'] = movies['crew'].apply(get_director)

In [133]:
movies['genres'] = movies['genres'].apply(get_genre_keywords)

In [134]:
movies['overview'] = movies['overview'].apply(lambda x: str(x).lower())

In [135]:
movies['tagline'] = movies['tagline'].apply(lambda x: str(x).lower())

In [136]:
movies['keywords'] = movies['keywords'].apply(get_genre_keywords)

In [137]:
movies = movies[['movie_id','title','overview','tagline','genres','keywords','cast','crew']]

In [138]:
movies

,movie_id,title,overview,tagline,genres,keywords,cast,crew
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di...",enter the world of pandora.,action adventure fantasy science fiction,culture clash future space war space colony s...,samworthington zoesaldana sigourneyweaver,jamescameron
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha...","at the end of the world, the adventure begins.",adventure fantasy action,ocean drug abuse exotic island east india tra...,johnnydepp orlandobloom keiraknightley,goreverbinski
2,206647,Spectre,a cryptic message from bond’s past sends him o...,a plan no one escapes,action adventure crime,spy based on novel secret agent sequel mi6 br...,danielcraig christophwaltz léaseydoux,sammendes
3,49026,The Dark Knight Rises,following the death of district attorney harve...,the legend ends,action crime drama thriller,dc comics crime fighter terrorist secret iden...,christianbale michaelcaine garyoldman,christophernolan
4,49529,John Carter,"john carter is a war-weary, former military ca...","lost in our world, found in another.",action adventure science fiction,based on novel mars medallion space travel pr...,taylorkitsch lynncollins samanthamorton,andrewstanton
...,...,...,...,...,...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...,"he didn't come looking for trouble, but troubl...",action crime thriller,united states–mexico barrier legs arms paper ...,carlosgallardo jaimedehoyos petermarquardt,robertrodriguez
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...,a newlywed couple's honeymoon is upended by th...,comedy romance,,edwardburns kerrybishé marshadietlein,edwardburns
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic...",nan,comedy drama romance tv movie,date love at first sight narration investigat...,ericmabius kristinbooth crystallowe,scottsmith
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...,a new yorker in shanghai,,,danielhenney elizacoupe billpaxton,danielhsia


In [139]:
movies['info'] = movies['overview'] + " " +  movies['tagline'] + " " +  movies['genres'] + " " +  movies['keywords'] + " " +  movies['cast'] + " " +  movies['crew']

In [140]:
movies = movies.drop(columns=['overview','tagline','genres','keywords','cast','crew'])

In [141]:
movies

,movie_id,title,info
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [142]:
movies = movies.dropna(axis = 0)

In [143]:
cnt_vec = CountVectorizer(max_features = 5000, stop_words = "english")

In [144]:
vector = cnt_vec.fit_transform(movies['info']).toarray()

In [146]:
cos_sim = cosine_similarity(vector)

In [147]:
cos_sim.shape

(4779, 4779)

In [149]:
def recommender(movie):
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(cos_sim[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:10]:
        print(movies.iloc[i[0]].title)

In [157]:
recommender("The Godfather")

Blood Ties
The Godfather: Part II
10th & Wolf
Easy Money
Mi America
Deadfall
Mambo Italiano
Black Mass
The Iceman


In [159]:
pickle.dump(movies,open('movie_list.pkl','wb'))
pickle.dump(cos_sim,open('similarity.pkl','wb'))